In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.3.0
Running on TPU  ['10.68.88.226:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.68.88.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.68.88.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Layer, MaxPooling2D, Flatten, Dense, Conv2D, Input, BatchNormalization, MaxPool2D, Lambda, Concatenate, Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
import numpy as np
import random
import time
%load_ext tensorboard
import datetime
!rm -rf ./logs/ 

### Make The Dataset

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train / 255
x_test = x_test / 255

y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)

image_size = (32,32,3)

# flip images
x_train_flipped = np.flip(x_train, axis=3)
x_train = np.append(x_train, x_train_flipped, axis=0)
y_train = np.append(y_train, np.copy(y_train), axis=0)
x_train_flipped = None
print('flipped')

# mix it up
x_train, y_train = unison_shuffled_copies(x_train, y_train)
print('permuted')
print(x_train.shape, y_train.shape)

flipped
permuted
(100000, 32, 32, 3) (100000, 100)


In [ ]:
classes_train = []
classes_test = []

for i in range(100): 
  classes_train.append([])
  classes_test.append([])
for i in range(len(y_train)):
  classes_train[np.argmax(y_train[i])].append(x_train[i])
for i in range(len(y_test)):
  classes_test[np.argmax(y_test[i])].append(x_test[i])

In [ ]:
word_labels = '''beaver, dolphin, otter, seal, whale, 
aquarium fish, flatfish, ray, shark, trout, 
orchids, poppies, roses, sunflowers, tulips, 
bottles, bowls, cans, cups, plates, 
apples, mushrooms, oranges, pears, sweet peppers, 
clock, computer keyboard, lamp, telephone, television, 
bed, chair, couch, table, wardrobe, 
bee, beetle, butterfly, caterpillar, cockroach, 
bear, leopard, lion, tiger, wolf, 
bridge, castle, house, road, skyscraper, 
cloud, forest, mountain, plain, sea, 
camel, cattle, chimpanzee, elephant, kangaroo, 
fox, porcupine, possum, raccoon, skunk, 
crab, lobster, snail, spider, worm, 
baby, boy, girl, man, woman, 
crocodile, dinosaur, lizard, snake, turtle, 
hamster, mouse, rabbit, shrew, squirrel, 
maple, oak, palm, pine, willow, 
bicycle, bus, motorcycle, pickup truck, train, 
lawn-mower, rocket, streetcar, tank, tractor'''.replace('\n', '').split(', ')

In [ ]:
# for sigmoid, values are 0 when they don't correspond
def make_level(nums):
  ytrain = np.zeros((y_train.shape[0], len(nums)))
  ytest = np.zeros((y_test.shape[0], len(nums)))

  for i in range(y_train.shape[0]):
    arg = np.argmax(y_train[i])
    for x in range(len(nums)):
      if arg in nums[x]:
        ytrain[i][x] = 1.0

  for i in range(y_test.shape[0]):
    arg = np.argmax(y_test[i])
    for x in range(len(nums)):
      if arg in nums[x]:
        ytest[i][x] = 1.0
  
  return ytrain, ytest

In [ ]:
def make_custom_level(allNums, nums):
  ytrain = []
  ytest = []

  for i in range(y_train.shape[0]):
    arg = np.argmax(y_train[i])
    if arg not in allNums: continue
    y = np.zeros(len(nums))
    for x in range(len(nums)):
      if arg in nums[x]:
        y[x] = 1.0
    ytrain.append(y)

  for i in range(y_test.shape[0]):
    arg = np.argmax(y_test[i])
    if arg not in allNums: continue
    y = np.zeros(len(nums))
    for x in range(len(nums)):
      if arg in nums[x]:
        y[x] = 1.0
    ytest.append(y)
  
  return np.array(ytrain), np.array(ytest)

In [ ]:
def make_custom_x(allNums):
  xtrain = []
  xtest = []

  for i in range(x_train.shape[0]):
    arg = np.argmax(y_train[i])
    if arg not in allNums: continue
    xtrain.append(x_train[i])

  for i in range(x_test.shape[0]):
    arg = np.argmax(y_test[i])
    if arg not in allNums: continue
    xtest.append(x_test[i])
  
  return np.array(xtrain), np.array(xtest)

### Model

In [ ]:
l1 = 1e-4
l2 = 5e-4

def mpu(in_shape, root=False, end=False, **kwargs):
  inp = Input(shape=in_shape)
  
  pred = Conv2D(64, (3,3), padding="same", activation="relu")(inp)
  pred = Conv2D(64, (3,3), padding="same", activation="relu")(pred)
  pred = MaxPool2D()(pred)
  pred = Flatten()(pred)
  pred = Dense(1, activation="sigmoid")(pred)

  base = Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(inp)
  base = Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(base)
  base = BatchNormalization()(base)
  base = Dropout(0.5)(base)

  if root:
    return Model(inp, base, **kwargs)
  if end:
    return Model(inp, pred, **kwargs)

  return Model(inputs=inp, outputs=[pred, base], **kwargs)

In [ ]:
class ModelNode:
  def __init__(self, in_shape=None, ind=-1, name=None):
    if in_shape != None:
      self.node = mpu(in_shape, name=name)
    self.branches = []
    self.name = name
    self.ind = [ind] if type(ind) == int else ind
    self.parent = None

  def __call__(self, model):
    if type(model) == ModelNode: 
      model.branches.append(self)
      self.parent = model
      self.extendInd(self.ind)
      shape = model.node.outputs[1].shape[1:]
    elif hasattr(model, "output_shape"): shape = model.output_shape[1:]
    else: shape = model.shape[1:]
    self.node = mpu(shape, name=self.name)

    return self
  
  def extend(self, path, transfer=False, name=None):
    if len(path) == 1:
      if path[0] >= len(self.branches):

        x = ModelNode(name=name)(self)
        if transfer:
          x.node.set_weights(self.node.get_weights()) 
      else:
        x = ModelNode(name=name)(self)
        if transfer:
          x.node.set_weights(self.node.get_weights()) 
        self.branches.insert(path[0], x)
        self.branches.pop()
    else:
      b = path.pop(0)
      self.branches[b].extend(path, transfer, name)
  
  def extendInd(self, ind):
    if self.parent:
      self.parent.ind.extend(ind)
      self.parent.extendInd(ind)
  
  # def getFullTree(self, input):
  #   pred, ext = self.node(input)
  #   models = [pred]
  #   for i in self.branches:
  #     model = i.getFullTree(ext)
  #     models.extend(model.outputs)
  
  def getFullTree(self, input):
    pred, ext = self.node(input)
    levels = [[pred]]
    for i in self.branches:
      model = i.getFullTree(ext)
      for i in range(len(model)):
        if len(levels) <= i+1: levels.append([])
        levels[i+1].extend(model[i])
    
    return levels
  
  def getTreeFromPath(self, input, path):
    inp = Input(shape=input.shape[1:])
    pred, ext = self.node(inp)
    if len(path) == 0: return Model(inputs=inp, outputs=pred, name=self.name)

    branch = self.branches[path.pop(0)]
    model = branch.getTreeFromPath(ext, path)(ext)
    return Model(inputs=inp, outputs=model, name=self.name)

### Functions For Expanding, Printing, and Saving

In [ ]:
def printTree(tree, spaces=0):
  print(' '*spaces, end='')
  if spaces > 0: print('⮡ ', end="")
  print(tree.name)
  for i in tree.branches:
    if spaces == 0:
      printTree(i, spaces + 1)
    else:
       printTree(i, spaces + 3)

In [ ]:
class Design:
  def __init__(self, design):
    self.design = design if type(design) == list else [design]

  def inds(self):
    inds = []
    for i in range(len(self.design)):
      if type(self.design[i]) == int:
        inds.append(self.design[i])
      else:
        inds.extend(self[i].inds())
    return inds
  
  def isNextLevel(self):
    for i in self.design:
      if type(i) == list:
        return True
    return False

  def nextLevel(self):
    l = []
    for i in self.design:
      if type(i) == list:
        l.extend(i)
    return Design(l)

  def path(self, num):
    p = []
    for i in range(len(self.design)):
      if self.design[i] == num: return [i]
      elif type(self.design[i]) == list:
        d = self[i].path(num)
        if d != None:
          return [i] + d
    return None

  
  def __getitem__(self, num):
    return Design(self.design[num])
  
  def __len__(self):
    return len(self.design)

In [ ]:
treeInds = []
for i in range(0, 100, 5):
  treeInds.append(list(range(i, i+5)))
design = Design(treeInds)

ROUND 1 ENDS HERE

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0005, momentum=0.9)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


with tpu_strategy.scope():
  inp = Input(shape=(32,32,3))
  m = ModelNode(name="root")(inp)

  back = False
  ref = design
  while ref.design != []:
    for i in range(len(ref)):
      inds = ref[i].inds()
      m.extend(design.path(ref.design[i]), name=str(inds).replace(', ', '.')[1:-1])
    ref = ref.nextLevel()

  nums = []
  for i in range(0, 100, 5):
    nums.append(range(i, i+5))

  inp =  Input(shape=(32,32,3))
  outputs = m.getFullTree(inp)[1:]
  for i in range(len(outputs)):
    outputs[i] = Concatenate()(outputs[i])

  full = Model(inp, outputs, name="full")

In [ ]:
### if round 2 starts from file
with tpu_strategy.scope():
  full = tf.keras.models.load_model('/content/drive/My Drive/mpub_cifar100_pretrain.hdf5')

In [ ]:
nums = []
for i in range(0, 100, 5):
  nums.append(range(i, i+5))

with tpu_strategy.scope():
  full_train, full_test = make_level(nums)

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0005, momentum=0.9)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/best_full_model.hdf5', monitor='val_loss', verbose=1,
#     save_best_only=True, mode='min', period=1)
with tpu_strategy.scope():
  full.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
full.fit(x_train, [full_train, y_train], epochs=100, batch_size=64, validation_data=(x_test, [full_test, y_test]), callbacks=[es])
print('stopped early, done training')
full.save('/content/drive/My Drive/pretrain_and_full.hdf5')
print('done 2')

Epoch 1/100
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/1563 [..............................] - ETA: 3:53 - loss: 14.1730 - concatenate_44_loss: 3.8772 - concatenate_45_loss: 5.3457 - concatenate_44_accuracy: 0.1016 - concatenate_45_accuracy: 0.0938WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0411s vs `on_train_batch_end` time: 0.2483s). Check your callbacks.


1563/1563 [==============================] - ETA: 0s - loss: 9.9793 - concatenate_44_loss: 2.7772 - concatenate_45_loss: 2.3822 - concatenate_44_accuracy: 0.1592 - concatenate_45_accuracy: 0.4062WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0259s vs `on_test_batch_end` time: 0.0941s). Check your callbacks.


1563/1563 [==============================] - 1066s 682ms/step - loss: 9.9793 - concatenate_44_loss: 2.7772 - concatenate_45_loss: 2.3822 - concatenate_44_accuracy: 0.1592 - concatenate_45_accuracy: 0.4062 - val_loss: 11.9745 - val_concatenate_44_loss: 2.9183 - val_concatenate_45_loss: 4.3774 - val_concatenate_44_accuracy: 0.1357 - val_concatenate_45_accuracy: 0.1557
Epoch 2/100
1563/1563 [==============================] - 470s 301ms/step - loss: 8.7455 - concatenate_44_loss: 2.5157 - concatenate_45_loss: 1.6931 - concatenate_44_accuracy: 0.2352 - concatenate_45_accuracy: 0.5397 - val_loss: 11.4591 - val_concatenate_44_loss: 3.2733 - val_concatenate_45_loss: 3.7909 - val_concatenate_44_accuracy: 0.1142 - val_concatenate_45_accuracy: 0.2679
Epoch 3/100
1563/1563 [==============================] - 479s 306ms/step - loss: 8.0341 - concatenate_44_loss: 2.3434 - concatenate_45_loss: 1.4343 - concatenate_44_accuracy: 0.2869 - concatenate_45_accuracy: 0.6001 - val_loss: 10.7883 - val_concatena

### Test The Networks

In [ ]:
### if round 2 starts from file
with tpu_strategy.scope():
  full = tf.keras.models.load_model('/content/drive/My Drive/full.hdf5')
full.load_weights('/content/drive/My Drive/best_full_model.hdf5'))

In [21]:
# # all = Model(imp, [coarse, interM, fine])
import copy
raw = full.predict(x_test)
tests = copy.deepcopy(raw)

In [22]:
abovep1 = 0
for i in tests[0]:
  if np.argmax(i) > 0:
    abovep1 += 1
print(abovep1, "/", len(tests[0]), '(', str(100*abovep1/len(tests[0])) + '%', ')')

9725 / 10000 ( 97.25% )


full

In [23]:
correct = 0
for i in range(len(y_test[1])):
  if np.argmax(tests[1][i]) == np.argmax(y_test[i]):
    correct += 1
print(correct / len(y_test[1]))

0.39


Top 3

In [27]:
correct = 0
for i in range(len(tests[1])):
  pred = list(reversed(np.argsort(tests[1][i])))
  if np.argmax(y_test[i]) in pred[:5]:
    correct += 1
print(correct / len(tests[1]))

0.6012


single

In [28]:
correct = 0
for i in range(len(y_test[1])):
  l1 = np.argmax(tests[0][i])
  l2 = np.argmax(tests[1][i][l1*5 : l1*5 + 5]) + l1*5
  if l2 == np.argmax(y_test[i]):
    correct += 1
print(correct / len(y_test[1]))

0.24


conditional

In [29]:
for threshold in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
  correct = 0
  tests = copy.deepcopy(raw)
  for i in range(len(y_test[1])):
    for j in range(len(tests[0][i])):
      if tests[0][i][j] < threshold:
        tests[0][i][j] = 0
    for j in range(len(tests[1][i])):
      if tests[0][i][j//5] == 0:
        tests[1][i][j] = 0
        continue
      if tests[1][i][j] < threshold:
        tests[1][i][j] = 0
    
    if np.argmax(tests[1][i]) == np.argmax(y_test[i]):
      correct += 1
  print(str(threshold) + ":", correct / len(y_test[1]))

1e-06: 0.39
1e-05: 0.4
0.0001: 0.38
0.001: 0.34
0.01: 0.23
0.1: 0.16
